In [2]:
import pandas as pd
import numpy as np
import torch  # Add this import
import torch.nn as nn
import torch.optim as optim

In [3]:
i = input('請輸入要讀取的資料編號(1~17):')
while i.isdigit() == False or int(i) < 1 or int(i) > 17:
    i = input('輸入錯誤，請重新輸入:')
    
train_file_paths_single = f'36_TrainingData/L{i}_Train.csv'#讀取單一資料
df_data = pd.read_csv(train_file_paths_single)
print(f'已讀取第{i}筆資料')

已讀取第1筆資料


In [9]:
# Move model and data to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
# Make numpy values easier to read.
np.set_printoptions(precision=3, suppress=True)
aicup_train = pd.read_csv(f'36_TrainingData/L{i}_Train_analysised.csv')
aicup_train.drop(columns=['Unnamed: 0',"DateTime"], inplace=True)
aicup_train_features = aicup_train.drop(columns=["Power(mW)"])
aicup_train_target = aicup_train["Power(mW)"]
# Define the model
class PredictModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(PredictModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.fc4 = nn.Linear(hidden_size, hidden_size)
        self.fc5 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

input_size = aicup_train_features.shape[1]
hidden_size = 64
output_size = 1

predict_model = PredictModel(input_size, hidden_size, output_size).to(device)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(predict_model.parameters(), lr=0.001)


Using device: cuda


In [10]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(aicup_train_features, aicup_train_target, test_size=0.2, random_state=42)

print(f"Training features shape: {X_train.shape}")
print(f"Testing features shape: {X_test.shape}")
print(f"Training target shape: {y_train.shape}")
print(f"Testing target shape: {y_test.shape}")

Training features shape: (81338, 4)
Testing features shape: (20335, 4)
Training target shape: (81338,)
Testing target shape: (20335,)


In [12]:
# Convert the training data to PyTorch tensors and move to the same device as the model
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1).to(device)

# Training loop

num_epochs = 5000

for epoch in range(num_epochs):
    # Forward pass
    outputs = predict_model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 1 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/5000], Loss: 10035.1055
Epoch [2/5000], Loss: 10013.2246
Epoch [3/5000], Loss: 9996.6406
Epoch [4/5000], Loss: 9991.3936
Epoch [5/5000], Loss: 9996.3350
Epoch [6/5000], Loss: 10005.1680
Epoch [7/5000], Loss: 10009.4492
Epoch [8/5000], Loss: 10006.6436
Epoch [9/5000], Loss: 9997.0029
Epoch [10/5000], Loss: 9986.3525
Epoch [11/5000], Loss: 9977.8867
Epoch [12/5000], Loss: 9974.2393
Epoch [13/5000], Loss: 9974.4854
Epoch [14/5000], Loss: 9976.5469
Epoch [15/5000], Loss: 9978.5859
Epoch [16/5000], Loss: 9978.4023
Epoch [17/5000], Loss: 9975.7363
Epoch [18/5000], Loss: 9970.8936
Epoch [19/5000], Loss: 9965.4746
Epoch [20/5000], Loss: 9960.5684
Epoch [21/5000], Loss: 9956.6191
Epoch [22/5000], Loss: 9953.7744
Epoch [23/5000], Loss: 9951.9922
Epoch [24/5000], Loss: 9950.9980
Epoch [25/5000], Loss: 9950.3818
Epoch [26/5000], Loss: 9949.9326
Epoch [27/5000], Loss: 9949.3164
Epoch [28/5000], Loss: 9948.4092
Epoch [29/5000], Loss: 9947.0205
Epoch [30/5000], Loss: 9945.2139
Epoch [31/5000

In [ ]:
import torch

# Convert the testing data to PyTorch tensors and move to the same device as the model
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1).to(device)

# Set the model to evaluation mode
predict_model.eval()

# Forward pass on the test set
with torch.no_grad():
    test_outputs = predict_model(X_test_tensor)
    test_loss = criterion(test_outputs, y_test_tensor)

print(f'Test Loss: {test_loss.item():.4f}')